In [2]:
using TimerOutputs
using GLMakie

include("CayleyVerify.jl")
include("DeepPoly.jl")

function dorefa_to_staircase(k::Int)
    n = 2^k - 1
    slopes = zeros(n+1)
    breakpoints = [-Inf]
    for i in 1:n
        push!(breakpoints, (2*i-1)/n - 1)
    end
    push!(breakpoints, Inf)
    
    constant_terms = [-1.0]
    for i in 1:n
        push!(constant_terms, -1.0 + 2*i/n)
    end
    return StaircaseFunction(breakpoints, slopes, constant_terms)
end

function predict(neural_net, img)
    num_layers = length(neural_net.weights)
    a = img'
    for i in 1:num_layers
        a = a * neural_net.weights[i] + neural_net.biases[i]'
        if i <= num_layers -1
            a = [eval(neural_net.activation[i], a[j]) for j in 1:length(a)]'
        end
    end
    output = a'
    return findmax(output)[2]
end
function reshape(img_array)
    n = Int(sqrt(length(img_array)))
    return Base.reshape(img_array, (n, n))
end

reshape (generic function with 1 method)

In [3]:
using Pickle
using Suppressor
using Printf
function read_model(model_path, f)
    net_from_pickle = Pickle.load(open(model_path))
    activation = [f, f]
    neural_net = NeuralNetwork(net_from_pickle, activation)
    return neural_net, imgs, labels
end


read_model (generic function with 1 method)

In [4]:
#println("model loaded")
raw_imgs = Pickle.load(open("./imgs/MNIST_images-for-verification"))
imgs = []
for img in raw_imgs
    img = vcat([w' for w in img] ...)
    img = vcat(img'...)
    push!(imgs, img)
end
labels = Pickle.load(open("./imgs/MNIST_labels-for-verification", "r+"))
labels = [l+1 for l in labels]

print("images loaded\n")
write("results.txt", "Model\tUpper Bound\tLower Bound\tTime\tEpsilon\n")

images loaded


43

# Cayley Embedding Solution

In [5]:
using Dates

function collect_files(directory)
    return [joinpath(directory, f) for f in readdir(directory) if isfile(joinpath(directory, f))]
end

# Usage

models = collect_files("./models")
println(models)



for eps in [0.008, 0.016, 0.024, 0.032]
    net_from_pickle = Pickle.load(open("./models/MNIST-DoReFa3_Dense256-Dense256.pkl"))
    f = dorefa_to_staircase(3)
    activation = [f, f]
    neural_net = NeuralNetwork(net_from_pickle, activation)
    start_time = now()
    upper_bound = 150
    lower_bound = 0
    count = 1
    for (img, label) in zip(imgs, labels)
        ## can run these individually since it's 1:1 image to target_attack
        @printf("upper bound: %d, lower bound: %d \n", upper_bound, lower_bound)
        @printf("Verify %d-th image \n", count)
        vulnerable = false
        for target_label in 1:10
            if target_label != label
                @suppress begin
                    opt_val, opt_sol_x, opt_sol_z = target_attack(neural_net, img, label, target_label, eps)
                    if opt_val > 0
                        vulnerable = true
                        adv_img = [opt_sol_x[1, j] for j in 1:784]
                        pred = predict(neural_net, adv_img)
                        if pred != label
                            upper_bound -= 1
                            break
                        end
                    end
                end
            end
        end
        if vulnerable == false
            lower_bound += 1
        end
        count += 1
    end
    end_time = now()
    elapsed_time = Dates.value(end_time - start_time) / (1000 * 60)
    open("results.txt", "a") do file
        write(file, "./models/MNIST-DoReFa3_Dense256-Dense256.pkl\t$upper_bound\t$lower_bound\t$elapsed_time\t$eps\n")
    end
end




["./models/MNIST-BAN_Dense100-Dense100-Dense100.pkl", "./models/MNIST-BAN_Dense100-Dense100.pkl", "./models/MNIST-BAN_Dense256-Dense256.pkl", "./models/MNIST-DoReFa2_Dense256-Dense256-Dense256.pkl", "./models/MNIST-DoReFa2_Dense256-Dense256.pkl", "./models/MNIST-DoReFa3_Dense256-Dense256-Dense256.pkl", "./models/MNIST-DoReFa3_Dense256-Dense256.pkl", "./models/MNIST-DoReFa4_Dense256-Dense256-Dense256.pkl", "./models/MNIST-DoReFa4_Dense256-Dense256.pkl", "./models/MNIST-DoReFa5_Dense256-Dense256.pkl", "./models/MNIST-DoReFa6_Dense256-Dense256.pkl", "./models/MNIST-DoReFa7_Dense256-Dense256.pkl"]
upper bound: 150, lower bound: 0 
Verify 1-th image 
upper bound: 150, lower bound: 1 
Verify 2-th image 
upper bound: 150, lower bound: 2 
Verify 3-th image 
upper bound: 150, lower bound: 3 
Verify 4-th image 
upper bound: 150, lower bound: 4 
Verify 5-th image 
upper bound: 150, lower bound: 5 
Verify 6-th image 
upper bound: 150, lower bound: 6 
Verify 7-th image 
upper bound: 150, lower boun

In [5]:
using Dates

function collect_files(directory)
    return [joinpath(directory, f) for f in readdir(directory) if isfile(joinpath(directory, f))]
end

# Usage

models = collect_files("./models")
println(models)
results = [["Model", "Upper Bound", "Lower Bound", "Time", "Epsilon"]]


for eps in [0.008, 0.016, 0.024, 0.032]
    net_from_pickle = Pickle.load(open("./models/MNIST-DoReFa2_Dense256-Dense256.pkl"))
    f = dorefa_to_staircase(2)
    activation = [f, f]
    neural_net = NeuralNetwork(net_from_pickle, activation)
    start_time = now()
    upper_bound = 150
    lower_bound = 0
    count = 1
    for (img, label) in zip(imgs, labels)
        @printf("upper bound: %d, lower bound: %d \n", upper_bound, lower_bound)
        @printf("Verify %d-th image \n", count)
        vulnerable = false
        for target_label in 1:10
            if target_label != label
                @suppress begin
                    opt_val, opt_sol_x, opt_sol_z = target_attack(neural_net, img, label, target_label, eps)
                    if opt_val > 0
                        vulnerable = true
                        adv_img = [opt_sol_x[1, j] for j in 1:784]
                        pred = predict(neural_net, adv_img)
                        if pred != label
                            upper_bound -= 1
                            break
                        end
                    end
                end
            end
        end
        if vulnerable == false
            lower_bound += 1
        end
        count += 1
    end
    end_time = now()
    elapsed_time = Dates.value(end_time - start_time) / (1000 * 60)
    open("results.txt", "a") do file
        write(file, "./models/MNIST-DoReFa2_Dense256-Dense256.pkl\t$upper_bound\t$lower_bound\t$elapsed_time\t$eps\n")
    end
end




["./models/MNIST-DoReFa2_Dense256-Dense256-Dense256.pkl", "./models/MNIST-DoReFa2_Dense256-Dense256.pkl", "./models/MNIST-DoReFa3_Dense256-Dense256-Dense256.pkl", "./models/MNIST-DoReFa3_Dense256-Dense256.pkl", "./models/MNIST-DoReFa4_Dense256-Dense256-Dense256.pkl", "./models/MNIST-DoReFa4_Dense256-Dense256.pkl", "./models/MNIST-DoReFa5_Dense256-Dense256.pkl", "./models/MNIST-DoReFa6_Dense256-Dense256.pkl", "./models/MNIST-DoReFa7_Dense256-Dense256.pkl"]
upper bound: 150, lower bound: 0 
Verify 1-th image 
upper bound: 150, lower bound: 1 
Verify 2-th image 
upper bound: 150, lower bound: 2 
Verify 3-th image 
upper bound: 150, lower bound: 3 
Verify 4-th image 
upper bound: 150, lower bound: 4 
Verify 5-th image 
upper bound: 150, lower bound: 5 
Verify 6-th image 
upper bound: 150, lower bound: 6 
Verify 7-th image 
upper bound: 150, lower bound: 7 
Verify 8-th image 
upper bound: 150, lower bound: 8 
Verify 9-th image 
upper bound: 150, lower bound: 9 
Verify 10-th image 
upper bou

In [ ]:
using Dates

function collect_files(directory)
    return [joinpath(directory, f) for f in readdir(directory) if isfile(joinpath(directory, f))]
end

# Usage

models = collect_files("./models")
println(models)
results = [["Model", "Upper Bound", "Lower Bound", "Time", "Epsilon"]]


for eps in [0.008, 0.016, 0.024, 0.032]
    net_from_pickle = Pickle.load(open("./models/MNIST-DoReFa4_Dense256-Dense256.pkl"))
    f = dorefa_to_staircase(4)
    activation = [f, f]
    neural_net = NeuralNetwork(net_from_pickle, activation)
    start_time = now()
    upper_bound = 150
    lower_bound = 0
    count = 1
    for (img, label) in zip(imgs, labels)
        @printf("upper bound: %d, lower bound: %d \n", upper_bound, lower_bound)
        @printf("Verify %d-th image \n", count)
        vulnerable = false
        for target_label in 1:10
            if target_label != label
                @suppress begin
                    opt_val, opt_sol_x, opt_sol_z = target_attack(neural_net, img, label, target_label, eps)
                    if opt_val > 0
                        vulnerable = true
                        adv_img = [opt_sol_x[1, j] for j in 1:784]
                        pred = predict(neural_net, adv_img)
                        if pred != label
                            upper_bound -= 1
                            break
                        end
                    end
                end
            end
        end
        if vulnerable == false
            lower_bound += 1
        end
        count += 1
    end
    end_time = now()
    elapsed_time = Dates.value(end_time - start_time) / (1000 * 60)
    push!(results, ["./models/MNIST-DoReFa4_Dense256-Dense256.pkl", upper_bound,  lower_bound, elapsed_time, eps])
end




In [11]:
upper_bound

upper_bound (generic function with 2 methods)

In [12]:
lower_bound

lower_bound (generic function with 2 methods)

In [13]:
upper_bound

upper_bound (generic function with 2 methods)

In [14]:
lower_bound

lower_bound (generic function with 2 methods)

In [15]:
upper_bound

upper_bound (generic function with 2 methods)

In [16]:
lower_bound

lower_bound (generic function with 2 methods)

In [17]:
upper_bound

upper_bound (generic function with 2 methods)

In [18]:
lower_bound

lower_bound (generic function with 2 methods)

In [19]:
upper_bound

upper_bound (generic function with 2 methods)

In [20]:
lower_bound

lower_bound (generic function with 2 methods)

In [21]:
upper_bound

upper_bound (generic function with 2 methods)

In [1]:
net_from_pickle = Pickle.load(open("./models/MNIST-DoReFa3_Dense256-Dense256.pkl"))
f = dorefa_to_staircase(3)
activation = [f, f]
neural_net = NeuralNetwork(net_from_pickle, activation)
print("net loaded")

UndefVarError: UndefVarError: `Pickle` not defined

# Big-M Solution

In [7]:
upper_bound = 150
lower_bound = 0
count = 1
for (img, label) in zip(imgs, labels)
    @printf("Verify %d-th image \n", count)
    vulnerable = false
    for target_label in 1:10
        if target_label != label
            @suppress begin
                mip, _, _ = init_mip_deeppoly(neural_net, img, 0.024)
                last_layer = last(neural_net.weights)
                objective = zeros(10) # always 10 classes
                objective[target_label] = 1.0
                objective[label] = -1.0
                c = last_layer * objective

                num_layers = length(neural_net.weights)
                final_dim, output_dim = size(last_layer)
                @objective(mip, Max, sum(c[i]*mip[:x][num_layers, i] for i in 1:final_dim))
                optimize!(mip)
                opt_val = objective_value(mip)
                if opt_val > 0
                    vulnerable = true
                    break
                end
            end
        end
    end
    if vulnerable == false
        lower_bound += 1
    end
    count += 1
end

Verify 1-th image 
Verify 2-th image 
Verify 3-th image 
Verify 4-th image 
Verify 5-th image 
Verify 6-th image 
Verify 7-th image 
Verify 8-th image 
Verify 9-th image 
Verify 10-th image 
Verify 11-th image 
Verify 12-th image 
Verify 13-th image 
Verify 14-th image 
Verify 15-th image 
Verify 16-th image 
Verify 17-th image 
Verify 18-th image 
Verify 19-th image 
Verify 20-th image 
Verify 21-th image 
Verify 22-th image 
Verify 23-th image 
Verify 24-th image 
Verify 25-th image 
Verify 26-th image 
Verify 27-th image 
Verify 28-th image 


In [ ]:
lower_bound

In [ ]:
lower_bound

In [ ]:
lower_bound

In [ ]:
lower_bound

In [ ]:
lower_bound

In [ ]:
lower_bound